In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language
repo_path = "/home/haotian/Splitted_code"

loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".cpp"],
    # exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.CPP, parser_threshold=500),
)
documents = loader.load()
repo_path_2 = "/home/haotian/book"

loader2 = GenericLoader.from_filesystem(
    repo_path_2,
    glob="**/*",
    suffixes=[".txt"],
    # exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.CPP, parser_threshold=500),
)
documents2 = loader2.load()

/home/haotian/.local/lib/python3.8/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [3]:
documents = documents + documents2

[Document(page_content='void top(S a[N], S b[N], S c[N]) {\n\n#pragma HLS interface bram port = c\n#pragma HLS interface ap_memory port = a\n#pragma HLS aggregate variable = a compact = byte\n#pragma HLS aggregate variable = c compact = byte\n\ntop_label0:\n    for (int i = 0; i < N; i++) {\n#pragma HLS PIPELINE II = 3\n        c[i].q.m = a[i].q.m + b[i].q.m;\n        c[i].q.n = a[i].q.n - b[i].q.n;\n        c[i].q.o = a[i].q.o || b[i].q.o;\n        c[i].p = a[i].q.n;\n    }\n}\n', metadata={'source': '/home/haotian/Splitted_code/67.cpp', 'language': <Language.CPP: 'cpp'>}),
 Document(page_content='void diamond(vecOf16Words* vecIn, vecOf16Words* vecOut, int size) {\n// The depth setting is required for pointer to array in the interface.\n#pragma HLS INTERFACE m_axi port = vecIn depth = 32\n#pragma HLS INTERFACE m_axi port = vecOut depth = 32\n\n    hls::stream<vecOf16Words> c0, c1, c2, c3, c4, c5;\n    assert(size % 16 == 0);\n\n#pragma HLS dataflow\n    load(vecIn, c0, size);\n    com

In [3]:
len(documents)
from langchain_text_splitters import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CPP, chunk_size=1000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

121

In [4]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("flax-sentence-embeddings/st-codesearch-distilroberta-base")
import numpy as np
np.array(model.encode(texts[0].page_content).shape)

array([768])

In [5]:
txt = []
for text in texts:
    txt.append(np.array(model.encode(text.page_content)))

In [8]:
query = '''
void kernel_2mm(int ni,int nj,int nk,int nl,float alpha,float beta,float tmp[40][50],float A[40][70],float B[70][50],float C[50][80],float D[40][80])
{
  int i;
  int j;
  int k;
{
    for (i = 0; i < 40; i++) {
      for (j = 0; j < 50; j++) {
        tmp[i][j] = 0.0;
        
        for (k = 0; k < 70; ++k) {
          tmp[i][j] += alpha * A[i][k] * B[k][j];
        }
      }
    }
    for (i = 0; i < 40; i++) {
      for (j = 0; j < 80; j++) {
        D[i][j] *= beta;
        
        for (k = 0; k < 50; ++k) {
          D[i][j] += tmp[i][k] * C[k][j];
        }
      }
    }
  }
}'''
# top k search
import numpy as np
k = 5
query_embed = model.encode(query)
score = []
for t in txt:
    score.append(query_embed.T @ t )
    # score.append(np.linalg.norm(query_embed - t))
indexed_list = sorted(enumerate(score), key=lambda x: x[1], reverse= True)
# indexed_list = sorted(enumerate(score), key=lambda x: x[1], reverse= False)
sorted_indices = [index for index, value in indexed_list]
top_k_index = sorted_indices[:k]
top_k_doc = []
for i in top_k_index:
    top_k_doc.append(texts[i])
top_k_doc
r_doc = []
for t in top_k_doc:
    r_doc.append(t.page_content)
r_doc = r_doc[:2]

In [9]:
prompt = f'''Here are some examples of matrix multiplication code with pragma hls lines added{",".join(r_doc)},
Now try your best to replace <FILL> with Pragma HLS lines:{query}'''
prompt

'Here are some examples of matrix multiplication code with pragma hls lines addedvoid funcA(data_t* in, data_t* out1, data_t* out2) {\nLoop0:\n    for (int i = 0; i < N; i++) {\n#pragma HLS pipeline rewind\n#pragma HLS unroll factor = 2\n        data_t t = in[i] * 3;\n        out1[i] = t;\n        out2[i] = t;\n    }\n}\n\nvoid funcB(data_t* in, data_t* out) {\nLoop0:\n    for (int i = 0; i < N; i++) {\n#pragma HLS pipeline rewind\n#pragma HLS unroll factor = 2\n        out[i] = in[i] + 25;\n    }\n}\n\nvoid funcC(data_t* in, data_t* out) {\nLoop0:\n    for (data_t i = 0; i < N; i++) {\n#pragma HLS pipeline rewind\n#pragma HLS unroll factor = 2\n        out[i] = in[i] * 2;\n    }\n}\n\nvoid funcD(data_t* in1, data_t* in2, data_t* out) {\nLoop0:\n    for (int i = 0; i < N; i++) {\n#pragma HLS pipeline rewind\n#pragma HLS unroll factor = 2\n        out[i] = in1[i] + in2[i] * 2;\n    }\n},void funcA(data_t* in, data_t* out1, data_t* out2) {\n#pragma HLS inline off\nLoop0:\n    for (int i 